### Install requirements

First, run the cells below to install the requirements:

In [1]:
import os
# from pprint import pprint
# import json

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login

from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/asif/miniconda3/envs/finetune/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "/home/asif/llm/llama-2-7b-model"
finetuned_model = "/home/asif/llm/medAlpaca/medLlama-2-finetuned"
model = AutoModelForCausalLM.from_pretrained(
    # config.base_model_name_or_path,
    finetuned_model,
    return_dict=True,
    device_map="auto",
    trust_remote_code=True,
)
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.99s/it]


## Inference

You can then directly use the trained model or the model that you have loaded from the 🤗 Hub for inference as you would do it usually in `transformers`.

In [18]:
generation_config = model.generation_config
generation_config.max_new_tokens = 400
generation_config_temperature = 0.3
generation_config.top_p = 0.95
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config_eod_token_id = tokenizer.eos_token_id
generation_config.repetition_penalty = 1.4 

In [4]:
DEVICE = "cuda"

In [5]:
def generate_response(question: str) -> str:
    prompt = f"""
    {question}
    """.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)

    assistant_start = 'Chatbot:'
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start):].strip()


In [20]:
prompt = """Instruction: A medical assistant chatbot and a user is having friendly conversation. Chatbot is providing medical advice and answers medical related queries. 
input: What should I do if I'am feeling fever?
output: """
print(generate_response(prompt))

tion: A medical assistant chatbot and a user is having friendly conversation. Chatbot is providing medical advice and answers medical related queries. 
input: What should I do if I'am feeling fever?
output: If you have any of the following symptoms, call your doctor immediately or go to an emergency room as soon as possible after reading this list : Fever that does not come down with acetaminophen (Tylenol) in two days Severe headache Dizziness Shortness of breath Rapid pulse Fast heart rate Confusion Nausea Vomiting Diarrhea Stiff neck Muscle aches Joint pain


In [21]:
prompt = """role: A medical assistant chatbot and a user is having friendly conversation. Chatbot is providing medical advice and answers medical related queries. Chatbot keeps its answer short and factual. Provide only one response.
user: Doctor, I think I've been poisoned. I drank some ethylene glycol by mistake. What should I do?
chatbot:
"""
print(generate_response(prompt))

medical assistant chatbot and a user is having friendly conversation. Chatbot is providing medical advice and answers medical related queries. Chatbot keeps its answer short and factual. Provide only one response.
user: Doctor, I think I've been poisoned. I drank some ethylene glycol by mistake. What should I do?
chatbot: Ethanol (ethane-1 -hydroxy) can be used to treat the symptoms of alcohol withdrawal in patients who have stopped drinking for 3 days or more but are experiencing severe tremor/shakiness; anxiety ; hallucinations , seizures .


In [22]:
prompt = """instruction: A medical assistant chatbot and a user is having friendly conversation. Chatbot is providing medical advice and answers medical related queries. Chatbot keeps its answer short and factual. Provide only one response.
input: Doctor, I think I've been poisoned. I drank some ethylene glycol by mistake. What should I do?
output:
"""
print(generate_response(prompt))

tion: A medical assistant chatbot and a user is having friendly conversation. Chatbot is providing medical advice and answers medical related queries. Chatbot keeps its answer short and factual. Provide only one response.
input: Doctor, I think I've been poisoned. I drank some ethylene glycol by mistake. What should I do?
output: Ethanol (ethane-1 -hydroxy) can be used to treat the symptoms of acute alcohol intoxication such as nausea vomiting diarrhea hypotension respiratory depression central nervous system stimulation or cardiac arrhythmias .


In [23]:
prompt = """instruction: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
input: I am suffering from fever, what should I do?
output: """ 
print(generate_response(prompt))

tion: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
input: I am suffering from fever, what should I do?
output: Fever is the body’s natural response when it detects an infection or foreign substance that needs removal by white blood cells (WBC). It can be caused due to many reasons like viral illnesses such as influenza virus; bacterial diseases including strep throat & urinary tract infections etc.; parasitic infestations of malaria mosquito biting insects causing dengue/yellow fevers ; autoimmune disorders where WBcs attack healthy tissues leading to rheumatoid arthritis / lupus erythematosus .


In [24]:
prompt = """role: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: I am suffering from fever, what should I do?
chatbot: """ 
print(generate_response(prompt))

ou are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: I am suffering from fever, what should I do?
chatbot: Hi! Welcome on Chat Doctor ! Your symptoms could be due an infectious disease or other causes like allergy etc... For this reason it is important performing blood tests in order to determine the cause of these signs (complete count with differential for inflammation parameters). Meanwhile take paracetamol if needed but avoid ibuprofen because can worsen kidney functioning . Hope my advice was helpful don’t hesitate contact me again anytime Best wishes


In [25]:
prompt = """role: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: I am vomiting and having a high temparature, what should I do?
chatbot: """ 
print(generate_response(prompt))

ou are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: I am vomiting and having a high temparature, what should I do?
chatbot: Hi welcome at Chat Doctor .com Vomitings with fever can be due many reasons like infection of stomach or intestines (gastroenteritis), food poisoning etc., so better consult doctor for examination as soon possible Hope this helps


In [26]:
prompt = """role: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: How to make noodles?
chatbot: """ 
print(generate_response(prompt))

ou are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: How to make noodles?
chatbot: Noodle is made from wheat flour or rice powder with water added in it for kneading the dough which can be boiled after shaping into different shapes like round flat long etc...


In [27]:
prompt = """Instruction: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries.
Input: How to make noodles?
Output: """ 
print(generate_response(prompt))

tion: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries.
Input: How to make noodles?
Output: Noodle is made from wheat flour or rice powder with water added in it by hand kneading method for 10 minutes then the dough was divided into small portions of equal size which were rolled out flat on board using rolling pin after that they cut lengthwise & width wise as per requirement finally boiled till softened


In [29]:
prompt = """role: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: I am feeling fever, what should I do? 
chatbot: """
print(generate_response(prompt))

ou are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: I am feeling fever, what should I do? 
chatbot: Fever is an increase in body temperature above normal range of about +38°C (102F). It can be caused by many different conditions such as viral or bacterial illnesses like malaria etc., medications that cause side effects on the liver causing rise in blood levels leading to hepatitis which causes jaundice with itching all over the skin called icterus .


In [30]:
prompt = """role: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. 
user: Doctor, I've been experiencing a condition called cryptorchidism. My testicles have not descended properly into the scrotum. What medical tests do I need to take? 
chatbot: """
print(generate_response(prompt))

ou are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. 
user: Doctor, I've been experiencing a condition called cryptorchidism. My testicles have not descended properly into the scrotum. What medical tests do I need to take? 
chatbot: HiT hanks for using ChatDoctorI really appreciate your concern about this problem of yours that is Cryptorhydism which means undescendence or incomplete descent in males at birth . This can be due many reasons like genetic disorders , chromosomal abnormalities etc., but it may require surgery if there any other associated problems with these organs as well... So please consult an Urologist who would examine urethra by cystoureterography (CU) / retrograde pyelogram(RPG), ultrasound scan & MRI imaging studies ... If needed he might advise further investigations too.... Hope my advice helps


In [31]:
prompt = """role: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short. 
user: Hi Doctor, I have a problem with my mouth. It's painful and there's swelling around it. It's difficult to eat or talk properly. What is the problem in my mouth and how to recover? 
chatbot: """
print(generate_response(prompt))

ou are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short. 
user: Hi Doctor, I have a problem with my mouth. It's painful and there's swelling around it. It's difficult to eat or talk properly. What is the problem in my mouth and how to recover? 
chatbot:Hello! Welcome on ChatDoctor ! Your symptoms could be due an infected gum (gingivitis) that has spread into deeper tissues of cheek causing inflammation/swollen lymph nodes . For this reason ,I would recommend consulting at dentist for clinic examination & radiological tests if needed(x ray). Meanwhile start using antibiotic like amoxycillin+clavulanate twice daily after meals along witChatDoctore gel locally over affected area thrice day.(if no allergies ). Hopefully these measures should help relieve from discomfort within few days but don’t hesitate contact again us whenever necessary 

In [34]:
prompt = """role: You are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: How to make noodles?
Medibot: """
print(generate_response(prompt))

ou are a medical chatbot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: How to make noodles?
Medibot: Hi there! I am MedBOT - A Medical Chat Doctor !!! If u have any query regarding health issues or want me 2 suggest sum1 for treatment of ur disease then plz type here...I shall be glad 2 helpu out with my best services......thanks n take care!!!


In [35]:
prompt = """role: You are a medical chatbot named medibot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: I am feeling fever, what should I do? 
medibot: """
print(generate_response(prompt))

ou are a medical chatbot named medibot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: I am feeling fever, what should I do? 
medibot: Hi! Welcome on Chat Doctor ! Your symptoms could be due by an infection or other causes like stress etc... For this reason it is important performing the following tests:- complete blood count for inflammation - PCR (polymerase chain reaction) test of viruses that cause flu.- throat swab culture if there was sore throats recently-. urine analysis with microscopy . If these results were normal then no need worrying about serious diseases but just take paracetamol as anti pyretic medications when needed because they can help reduce temperature without causing side effects such as liver damage ... Hope my advice has been helpful don’t hesitate contact me again whenever necessary Best wishes


In [36]:
prompt = """role: You are a medical chatbot named medibot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: How to make noodles? 
medibot: """
print(generate_response(prompt))

ou are a medical chatbot named medibot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: How to make noodles? 
medibot: Hi there! I am Chat Doctor . If this is an emergency please call the local Emergency Services number (such as "911"). Otherwise feel free asking me any health questions or concerns that bothering u at present time... What would be ur query ? Please type here.... Noodle making process can vary from region of origin but generally it involves mixing flour with water until dough forms then rolling out thin sheets which get cut into strips called vermicelli after drying in sunlight for few days..... Hope my advice was helpful ! Feel Free To Ask Anytime !! Have A Good Day!!


In [37]:
prompt = """role: You are a medical chatbot named medibot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: How to write a poem? 
medibot: """
print(generate_response(prompt))

ou are a medical chatbot named medibot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: How to write a poem? 
medibot: Poetry is an art form that uses language in creative ways for its rhythm or meaningful content rather than just the basic communication of facts like prose does (although it can be used as such). It may use patterns based on metering out syllables with stress accents at regular intervals; this patterned structure gives rise to poetic lines which have their own distinctive sound when read aloud by someone who knows how they should go together - something called "poetics". The word itself comes from Greek words meaning 'making', so there isn't really any one definition but instead many different types depending upon what kind(s) we want our readers/listeners experience while reading / listening through these works themselves!


In [38]:
prompt = """role: You are a medical chatbot named medibot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: I am feeling weak, what should I do? 
medibot: """
print(generate_response(prompt))

ou are a medical chatbot named medibot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: I am feeling weak, what should I do? 
medibot: Hi! Welcome on Chat Doctor ! Your symptoms could be due by anemia or thyroid dysfunction (hypothyroidism). For this reason it is necessary performing blood tests in order to investigate for hemoglobin level as well the function of thryoid gland hormones TSH , free t3 &t4 . If these test results would indicate any abnormality then further investigations like ultrasound scan may need according with specialist opinion. Hope my advice can help you today but if something more information needed don’t hesitate contacting again here at ChatDoctor Best wishes


# Good prompt: 

role: You are a medical chatbot named medibot. User wants to know some medical related queries, and you will answer them. You'll also provide medical related advices. But do not answer non-medical queries. Keep your answer short.
user: I am feeling weak, what should I do? 
medibot: 